## Submission 2 - Gradient Boosting Regression

I think the following code was used in the second submission, but I didn't take the time on making it clear and pretty since it's not in our final submission.

In [43]:
def read_files(lFilesPred, sFileLabels, sTypeLabel):
    """
    TODO 
    
    Keyword Arguments: 
    - lFilesPred: 
    - sFileLabels: 
    - sTypeLabels: 
    """
    
    # Read file labels (true labels)
    print("sFileLabels : ", sFileLabels)
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dID = {rows[0]:rows[1] for rows in reader} #participant ID
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dOnOff= {rows[0]:rows[2] for rows in reader} #on-off label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dDys={rows[0]:rows[3] for rows in reader} #dyskinesia label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dTrem={rows[0]:rows[4] for rows in reader} #tremor label
    
    # Training-testing data
    iNumFiles=len(lFilesPred)
    #mPredictions=[] #np.zeros((1,iNumFiles))
    #mPredictions=np.asarray(mPredictions)
    vPredIter=np.zeros((1,iNumFiles))
    vLabels=[] #true label
    vParID=[] # participant ID
    lDicts=[] 


    # We loop over the predictions files from the tsfresh and SVR
    # lDicts is a list of measurement_id and the predictions obtained 
    for sFilePred in lFilesPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (ivec-svr, boost...)
        lDicts.append(dPred)
    print('sTypeLabel : ', sTypeLabel)
    #label selection
    if sTypeLabel=='on_off':
        dLabels=dOnOff
    elif sTypeLabel=='tremor':
        dLabels=dTrem
    elif sTypeLabel=='dyskinesia':
        dLabels=dDys
    else:
        print('sTypeLabel undefined')
    
    print('dLabels')
    print(dLabels)
    
    #creation of the matrix containing prediction from all classifiers
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            #print(k)
            #print(dID[k])
            if dLabels[k]!='NA':
                vLabels.append(float(np.asarray(dLabels[k]))) #true labels
                vParID.append(float(np.asarray(dID[k]))) #participant ID
                vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
                for j in range(1, iNumFiles):
                    fPred=lDicts[j].get(k)
                if fPred:
                    vPredIter[0,j]=float(np.asarray(fPred))
                else:
                    print(['Unkwnown key:' + k])
                    vPredIter[0,j]=float(np.asarray(lDicts[0][k]))
                    
            if bEnter==1:
                mPredictions=vPredIter
                bEnter=0
            else:
                mPredictions=np.append(mPredictions,vPredIter,axis=0)
         
    print('mPredictions and vLabels')
    print(mPredictions.shape)
    print(len(vLabels))
    # Random forest training - regression
    
    #scores = cross_val_score(clf, mPredictions, vLabels, cv=15)
    #print('Cross-validation score:')
    #print(scores.mean())
    return mPredictions, vLabels

In [7]:
def RandomForest_fusion(lFilesPred, sFileLabels, iEstimators, sTypeLabel, nudge, rLR, iMD,iRS):

    mPredictions, vLabels = read_files(lFilesPred, sFileLabels, sTypeLabel)
    
    clf=GradientBoostingRegressor(n_estimators=iEstimators, learning_rate=rLR, max_depth=iMD,\
                                      random_state=iRS, loss='ls').fit(mPredictions, vLabels)
    vNewPred=clf.predict(mPredictions)
    vRes1=mPredictions[:,[0]]
    vRes2=mPredictions[:,[1]]
    
    # we will include the testing data here
    return (vNewPred, vParID, vLabels, vRes1, vRes2, clf)
#print(mPredictions)

In [8]:
def randomForest_predTest(lTestPred, clf, sDirOut,sName):
    
    lDicts=[] #Will contain the two test predictions
    lmesID=[]
    iNumFiles=len(lTestPred)
    vPredIter=np.zeros((1,iNumFiles))

    for sFilePred in lTestPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (ivec-svr, boost...)
        lDicts.append(dPred)
      
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            #print(k)
            #print(dID[k])
            lmesID.append(k)
            #vParID.append(float(np.asarray(dID[k]))) #participant ID
            vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
            for j in range(1, iNumFiles):
                fPred=lDicts[j].get(k)
            if fPred:
                vPredIter[0,j]=float(np.asarray(fPred))
            else:
                print(['Unkwnown key:' + k])
                vPredIter[0,j]=float(np.asarray(lDicts[0][k]))

            if bEnter==1:
                mPredictions=vPredIter
                bEnter=0
            else:
                mPredictions=np.append(mPredictions,vPredIter,axis=0)
         
    
    vTestPred=clf.predict(mPredictions)
    df = pd.DataFrame({'measurement_id': lmesID, sName:vTestPred})
    df.to_csv(sDirOut+'submissionCisPD'+sName+'.csv', index=False)
    return(lmesID, vTestPred)



### Dyskinesia

In [ ]:
#sFilePred1='/export/b15/nchen/BeatPD/cispd.kfold_prediction_on_off.csv'
sFilePred1='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sFilePred2='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sTypeLabel='on_off'
n_estimators=300
rLR=0.1
iMD=1
iRS=15
nudge=100
sFileLabels=sFilePred1

lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1,\
vRes2, clf = RandomForest_fusion (lFilesPred, sFileLabels, n_estimators, sTypeLabel, nudge, rLR, iMD,iRS)

print('Overall MSE Classif. 1 : ', get_final_score(vTrueLabels,np.array(vParID).astype(int), vRes1))
print('Overall MSE Classif. 2 : ', get_final_score(vTrueLabels, vParID, vRes2))
print('Overall MSE Fusion : ', get_final_score(vPredCross, vParIDCross,vTrueCross))




sTestPred1='/export/b15/nchen/BeatPD/new_features/submission/cis-pd.dyskinesia.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30/exp/ivec_450/resiVecSVR_Fold/On_off_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/'
sName='on_off'
#lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
#print(vTestPred)




### Tremor - Submission 2

In [ ]:
# tsfresh predictions file 
sFilePred1='/export/b15/nchen/BeatPD/new_features/kfold_prediction_tremor.csv'

# SVR predictions file 
sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_450/resiVecSVR_Fold_all/objs_450_kernel_linear_c_0.02_eps_0.1.csv'

# subchallenge 
sTypeLabel='tremor'

# RandomForest hyperparameter 
n_estimators=5

# Labels
sFileLabels='/export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'


lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1, vRes2, clf = RandomForest_fusion(lFilesPred,
                                                                           sFileLabels,
                                                                           n_estimators,
                                                                           sTypeLabel)

sTestPred1='/export/b15/nchen/BeatPD/submission/cis-pd.tremor.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_450/resiVecSVR_Fold/Tremor_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/export/c08/lmorove1/kaldi/egs/beatPDivec/jupyternotebooks/submissionFiles/'
sName='tremor'
lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
print(vTestPred)



### Dyskinesia - Submission 2

In [ ]:
#sFilePred1='/export/b15/nchen/BeatPD/cispd.kfold_prediction_dyskinesia.csv'
sFilePred1='/export/b15/nchen/BeatPD/new_features/kfold_prediction_dyskinesia.csv'
sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_dysk_auto/exp/ivec_500/resiVecSVR_Fold_all/objs_500_kernel_linear_c_0.002_eps_0.1.csv'
sTypeLabel='dyskinesia'
n_estimators=5
sFileLabels='/export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels, vRes1, vRes2, clf = RandomForest_fusion (lFilesPred, sFileLabels, n_estimators, sTypeLabel)

sTestPred1='/export/b15/nchen/BeatPD/submission/cis-pd.dyskinesia.csv'
sTestPred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_dysk_auto/exp/ivec_500/resiVecSVR_Fold/Dyskinesia_testing.csv'
lTestPred=[sTestPred1,sTestPred2]

sDirOut='/export/c08/lmorove1/kaldi/egs/beatPDivec/jupyternotebooks/submissionFiles/'
sName='z_dyskinesia'
lmesID, vTestPred = randomForest_predTest(lTestPred, clf, sDirOut, sName)
print(vTestPred)



# Other drafts

In [ ]:
%load_ext autoreload
%autoreload 2
from pca_everyone_svr_bpd import *

sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Training_Fold0/ivector.scp"
sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Testing_Fold0/ivector.scp"
iComponents=50

vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

In [ ]:
%load_ext autoreload
%autoreload 2
from pca_knn_bpd2 import *

sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Training_Fold0/ivector.scp"
sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Testing_Fold0/ivector.scp"
iComponents=50

vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np 
uniq_subjectid = np.unique(vTraiSubjectId)
enc = OneHotEncoder(handle_unknown='ignore').fit(uniq_subjectid.reshape(-1,1))
print(uniq_subjectid.reshape(-1,1).shape)
print(enc)
temp = enc.transform([[1004]]).toarray()
print(temp.shape)
temp2 = np.repeat(temp,61,axis=0)
temp2.shape
#temp3 = np.random.rand(61,50)
#temp4 = np.concatenate((temp3,temp2),axis=1)
#print(temp4.shape)

In [ ]:
%load_ext autoreload
%autoreload 2
from pca_knn_bpd2 import *

# Get Predictions for Per Patient SVR

## Tremor Best Config 

In [ ]:
for fold in ["0","1","2","3","4"]:
    sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_550/ivectors_Training_Fold"+fold+"/ivector.scp"
    sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_550/ivectors_Testing_Fold"+fold+"/ivector.scp"

    sOut="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto30/exp/ivec_550/resiVecPerPatientSVR_Fold"+fold

    iNeighbors=None

    for iComponent in [250]:#[50, 100, 150, 200, 250, 350, 450, 500, 550]:

        for fCValue in [20.0]:#[2e-13, 2e-07, 0.002, 0.2, 20.0]:
            vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponent)

            pca_knn_bpd2(sFileTrai, sFileTest, sOut,
                         iComponent,
                         iNeighbors,
                         sKernel='linear',
                         fCValue=fCValue,
                         fEpsilon='0.1',
                         bLabelNormalization=False,
                         bPatientPredictionsPkl=True)


    

## Dysk Best Config 

In [ ]:
for fold in ["0","1","2","3","4"]:
    sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_orig_auto60_400fl/exp/ivec_650/ivectors_Training_Fold"+fold+"/ivector.scp"
    sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_orig_auto60_400fl/exp/ivec_650/ivectors_Testing_Fold"+fold+"/ivector.scp"

    sOut="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_orig_auto60_400fl/exp/ivec_650/resiVecPerPatientSVR_Fold"+fold

    iNeighbors=None

    for iComponent in [100]:#[50, 100, 150, 200, 250, 350, 450, 500, 550]:

        for fCValue in [20.0]:#[2e-13, 2e-07, 0.002, 0.2, 20.0]:
            vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponent)

            pca_knn_bpd2(sFileTrai, sFileTest, sOut,
                         iComponent,
                         iNeighbors,
                         sKernel='linear',
                         fCValue=fCValue,
                         fEpsilon='0.1',
                         bLabelNormalization=False,
                         bPatientPredictionsPkl=True)


    

### Test with Mean Normalization SVR 

In [ ]:
for fold in ["0","1","2","3","4"]:
    sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Training_Fold"+fold+"/ivector.scp"
    sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Testing_Fold"+fold+"/ivector.scp"


    sOut="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/resiVecSVR_Fold"+fold

    iNeighbors=None

    vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

    iComponents=500
    pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.002, fEpsilon='0.1',
                 bLabelNormalization=True)


    # iComponents=400
    # pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.02, fEpsilon='0.01',
    #              bLabelNormalization=True)

### Per Patient - With Mean Normalization

In [ ]:
%load_ext autoreload
%autoreload 2

for fold in ["0","1","2","3","4"]:
    sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Training_Fold"+fold+"/ivector.scp"
    sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Testing_Fold"+fold+"/ivector.scp"


    sOut="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/resiVecSVR_Fold"+fold

    iNeighbors=None

    vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

    iComponents=500
    pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.002, fEpsilon='0.1',
                 bLabelNormalization=True)


    # iComponents=400
    # pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.02, fEpsilon='0.01',
    #              bLabelNormalization=True)
    
# sFilePath="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/"
# #sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
# bKnn=False
# bSVR=True
# bEveryoneSVR=False
# bPerSubject=False 
# sDatabase=None
# sSubchallenge=None


In [ ]:
from local.get_final_scores_accuracy import get_final_scores_SVR 
from local.get_final_scores_accuracy import get_final_scores_SVR_lowest_mse_for_subjectid

sFilePath="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/"
#sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
bKnn=False
bSVR=True
bEveryoneSVR=False
bPerSubject=True 
sDatabase='CIS'
sSubchallenge='dysk'

get_final_scores_SVR_lowest_mse_for_subjectid(sFilePath, bKnn, bSVR, bEveryoneSVR, bPerSubject, sDatabase, sSubchallenge)

# get_final_scores_SVR(sFilePath, bKnn, bSVR, bEveryoneSVR, bPerSubject, sDatabase, sSubchallenge)

### Test WITHOUT Mean Normalization

In [ ]:
for fold in ["0","1","2","3","4"]:
    sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Training_Fold"+fold+"/ivector.scp"
    sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Testing_Fold"+fold+"/ivector.scp"

    sOut="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/without_mean_norm/resiVecSVR_Fold"+fold

    iNeighbors=None

    iComponents=500
    vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

    pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.002, fEpsilon='0.1',
                 bLabelNormalization=False)

    # iComponents=400
    # pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.02, fEpsilon='0.01',
    #              bLabelNormalization=False)
    


In [ ]:
sFilePath="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/without_mean_norm/"
#sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
bKnn=False
bSVR=True
bEveryoneSVR=False
bPerSubject=True 
sDatabase='CIS'
sSubchallenge='dysk'

get_final_scores_SVR_lowest_mse_for_subjectid(sFilePath, bKnn, bSVR, bEveryoneSVR, bPerSubject, sDatabase, sSubchallenge)


#get_final_scores_SVR(sFilePath, bKnn, bSVR, bEveryoneSVR, bPerSubject, sDatabase, sSubchallenge)

In [ ]:
%load_ext autoreload
%autoreload 2

from local.get_final_scores_accuracy import get_final_scores_SVR 

sFilePath="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/without_mean_norm/"
#sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
bKnn=False
bSVR=True
bEveryoneSVR=False
bPerSubject=False 
sDatabase=None
sSubchallenge=None

get_final_scores_SVR(sFilePath, bKnn, bSVR, bEveryoneSVR, bPerSubject, sDatabase, sSubchallenge)

### SVR testing around

In [ ]:


sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Training_Fold0/ivector.scp"
sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/dysk_noinact_auto30/exp/ivec_500/ivectors_Testing_Fold0/ivector.scp"
iComponents=400

sOut="/home/mpgill/BeatPD/BeatPD-CLSP-JHU/ResiVecSVR_Fold0"

iNeighbors=None

vTraiPCA, vLTrai, vTraiSubjectId, vTraiMeasurementId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)

pca_knn_bpd2(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.02, fEpsilon='0.01')


In [ ]:

%load_ext autoreload
%autoreload 2
from pca_knn_bpd import *

sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30_320fl/exp/ivec_400/"

sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/ivectors_Training_Fold0/ivector.scp"
sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/ivectors_Testing_Fold0/ivector.scp" 

sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30_320fl/exp/ivec_400/ivectors_Training_Fold0/ivector.scp"
sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30_320fl/exp/ivec_400/ivectors_Testing_Fold0/ivector.scp" 


iComponents=50

vTraiPCA, vLTrai, vTraiSubjectId, vTestPCA, vLTest, vTestSubjectId, vTestMeasurementId = pca(sFileTrai, sFileTest, iComponents)
print(vTraiPCA.shape)

# /home/mpgill/BeatPD/BeatPD-CLSP-JHU/pca_everyone_svr_bpd.py 

# sFileTrai="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Training_Fold0/ivector.scp"
# sFileTest="/export/c08/lmorove1/kaldi/egs/beatPDivec/v1_autoenc/exp/ivec_100/ivectors_Testing_Fold0/ivector.scp"


sOut="/home/mpgill/test/"
sKernel='linear' 
fCValue=0.00000000200000 
fEpsilon=0.1
iNeighbors=None

pca_knn_bpd(sFileTrai, sFileTest, sOut, iComponents, iNeighbors, sKernel='linear', fCValue=0.02, fEpsilon='0.01')

In [ ]:
%load_ext autoreload
%autoreload 2

from get_final_scores_accuracy import get_final_scores_SVR 

sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30_320fl/exp/ivec_400/"
#sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
bKnn=False
bSVR=True
bEveryoneSVR=False

get_final_scores_SVR(sFilePath, bKnn, bSVR, bEveryoneSVR)

In [ ]:
%load_ext autoreload
%autoreload 2

from get_final_scores_accuracy import get_final_scores_SVR_lowest_mse_for_subjectid 

#sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30_320fl/exp/ivec_400/"
sFilePath="/export/c08/lmorove1/kaldi/egs/beatPDivec/trem_noinact_auto60_400fl/exp/ivec_400/"
bKnn=False
bSVR=True
bEveryoneSVR=False

#get_final_scores_SVR(sFilePath, bKnn, bSVR, bEveryoneSVR)

get_final_scores_SVR_lowest_mse_for_subjectid(sFilePath, bKnn, bSVR, bEveryoneSVR)

In [ ]:
salut = ['100001_test_5163afe8-a6b0-4ea4-b2ba-9b4501dd5912_10040',
 '100002_test_9152519b-4b57-43be-963c-dd7218495001_10040',
 '100003_test_c7312d73-cb34-4025-b8b8-5299b4033e2f_10040',
 '100004_test_cc730391-146b-420f-9255-c3185061f178_10040',
 '100005_test_cc7b822c-e310-46f0-a8ea-98c95fdb67a1_10041',
 '100006_test_5cf68c8e-0b7a-4b73-ad4f-015c7a20fb5a_10041',
 '100007_test_19a3e9ea-fce1-40b7-9457-2618970beb7b_10041',
 '100008_test_e2973da8-1250-4a7c-98d5-b165570a8aeb_10041',
 '100009_test_dc90dc36-b4e5-43ec-b3e8-47c39c763c71_10041']

salut2= ['100001_test_5163afe8-a6b0-4ea4-b2ba-9b4501dd5912_hbv0401',
 '100002_test_9152519b-4b57-43be-963c-dd7218495001_hbv0401',
 '100003_test_c7312d73-cb34-4025-b8b8-5299b4033e2f_hbv0401',
 '100004_test_cc730391-146b-420f-9255-c3185061f178_hbv0401',
 '100005_test_cc7b822c-e310-46f0-a8ea-98c95fdb67a1_hbv0411',
 '100006_test_5cf68c8e-0b7a-4b73-ad4f-015c7a20fb5a_hbv0411',
 '100007_test_19a3e9ea-fce1-40b7-9457-2618970beb7b_hbv0411',
 '100008_test_e2973da8-1250-4a7c-98d5-b165570a8aeb_hbv0412',
 '100009_test_dc90dc36-b4e5-43ec-b3e8-47c39c763c71_hbv0412']


sPattern=r'(?<=[_])(1[0-9]{3}(?<=\d)(?!=$)|hbv[0-9]{3})'

vTraiSubjectId = [re.findall(sPattern, fileName)[0] for fileName in salut]

# vTraiSubjectId = np.array(([x[-5:-1] for x in np.array(salut)]))
print(vTraiSubjectId)

vTraiSubjectId2 = [re.findall(sPattern, fileName)[0] for fileName in salut2]
print(vTraiSubjectId2)

In [ ]:
# import pandas as pd 
# import re


# def read_log_results_to_excel(folders, fileName):
#     for folder in folders:#'trem_noinact_auto30']:#,'trem_noinact_auto30_320fl','trem_noinact_auto30_240fl','trem_noinact_auto60_400fl']:
#         print(folder)
#         value = []
#         liVecDim = [350,400]#[350,400,450,500,550, 600, 650, 700]
#         for ivecDim in liVecDim:
#             sFilePath='/export/c08/lmorove1/kaldi/egs/beatPDivec/'+folder+'/exp/ivec_'+str(ivecDim)+'/'

#             #config_pattern = re.compile(r"Test Final score\s[:| ]\s*(\d*.\d*)")
# #             config_pattern = re.compile(r"GLOBAL WINNER PARAMETERS(.|\n)*\\KTest Final score\s[:| ]\s*(\d*.\d*)")


#             textfile = open(sFilePath+fileName)
#             filetext = textfile.read()
#             textfile.close()
# #             print(filetext)
#             matches = re.findall(r"GLOBAL WINNER PARAMETERS(?:.|\n)*\\KTest Final score\s[:| ]\s*(\d*.\d*)",
#                                  filetext)
# #             matches = re.findall(r"GLOBAL WINNER PARAMETERS(.|\n)*\\KTest Final score\s[:| ]\s*(\d*.\d*)",
# #                                  filetext)
#             print(matches)
# #             with open(sFilePath+fileName) as f:
# #                 print(f)
# #                 match = config_pattern.search(line)
# #                 if match:
# #                     #print(match.groups()[0])
# #                     value.append(match.groups()[0])
# #                     #yield line.strip()

#         value = pd.DataFrame(value)
#         value = value.T
#         value.columns = liVecDim
#         display(value)

# folders=['on_off_noinact_auto30_320fl']
# fileName='globalAccuPerPatientSVR_Test.log'
# read_log_results_to_excel(folders, fileName)

# # folders=['on_off_noinact_auto30_320fl']
# # fileName='globalAccuEveryoneSVR_Test.log'
# # read_log_results_to_excel(folders, fileName)

In [ ]:
# import pandas as pd 
# import re


# def read_log_results_to_excel(folders, fileName):
#     for folder in folders:#'trem_noinact_auto30']:#,'trem_noinact_auto30_320fl','trem_noinact_auto30_240fl','trem_noinact_auto60_400fl']:
#         print(folder)
#         value = []
#         liVecDim = [350,400,450,500,550, 600, 650, 700]
#         for ivecDim in liVecDim:
#             sFilePath='/export/c08/lmorove1/kaldi/egs/beatPDivec/'+folder+'/exp/ivec_'+str(ivecDim)+'/'

#             #config_pattern = re.compile(r"Test Final score\s[:| ]\s*(\d*.\d*)")
#             config_pattern = re.compile(r"Test Final score\s[:| ]\s*(\d*.\d*)")
# #             config_pattern = re.compile(r"GLOBAL WINNER PARAMETERS(?:.|\n)*\\KTest Final score\s[:| ]\s*(\d*.\d*)")
#             with open(sFilePath+fileName) as f:
#                 for line in f:
#                     match = config_pattern.search(line)
#                     if match:
#                         #print(match.groups()[0])
#                         value.append(match.groups()[0])
#                         #yield line.strip()

#         value = pd.DataFrame(value)
#         value = value.T
#         value.columns = liVecDim
#         display(value)

# folders=['on_off_noinact_auto30_320fl']
# fileName='globalAccuPerPatientSVR_Test.log'
# read_log_results_to_excel(folders, fileName)

# folders=['on_off_noinact_auto30_320fl']
# fileName='globalAccuEveryoneSVR_Test.log'
# read_log_results_to_excel(folders, fileName)

In [ ]:
import pandas as pd 
import re


def read_log_results_to_excel(folders, fileName):
    for folder in folders:#'trem_noinact_auto30']:#,'trem_noinact_auto30_320fl','trem_noinact_auto30_240fl','trem_noinact_auto60_400fl']:
        print(folder)
        value = []
        liVecDim = [350,450,500,550]
        for ivecDim in liVecDim:
            sFilePath='/export/c08/lmorove1/kaldi/egs/beatPDivec/'+folder+'/exp/ivec_'+str(ivecDim)+'/'
            print('Opening : ', sFilePath+fileName)
            textfile = open(sFilePath+fileName)
            filetext = textfile.read()
            textfile.close()
            
            
            result = re.findall(r"Test Final score\s[:| ]\s*(\d*.\d*)",filetext)
            print(result[len(result)-1])
            value.append(result[len(result)-1])
            
        value = pd.DataFrame(value)
        value = value.T
        value.columns = liVecDim
        display(value)

folders=['on_off_noinact_auto30']
fileName='globalAccuPerPatientSVR_Test.log'
read_log_results_to_excel(folders, fileName)

folders=['on_off_noinact_auto30_320fl']
fileName='globalAccuEveryoneSVR_Test.log'
read_log_results_to_excel(folders, fileName)

# Remove inactivity with max 

In [ ]:
def remove_inactivity_max(df_train_label):
    last_filtered_value = pd.Series(np.zeros(3), index=["X", "Y", "Z"])
    filtered_value = pd.Series(np.zeros(3), index=["X", "Y", "Z"])
    display(last_filtered_value)
    for idx in df_train_label.index:
        df_allo = []
        df_train_data = pd.read_csv(
            path_train_data + df_train_label["measurement_id"][idx] + ".csv"
        )

        # Get the absolute max value for X, Y, Z
        max_values = df_train_data.iloc[:, -3:].abs().max()

        # Compute what is 5% of that max
        thresold_energy = 5
        df_treshold = (max_values * thresold_energy) / 100

        # display(df_train_data)
        # Candidates are the frames where X, Y, Z are below that treshold (5% of the max)
        #         df_candidates = df_train_data[(df_train_data.X.abs() <= df_treshold.X) &
        #                                      (df_train_data.Y.abs() <= df_treshold.Y) &
        #                                      (df_train_data.Z.abs() <= df_treshold.Z)]
        # display(df_candidates)
        for idx2 in df_train_data.index:
            # print('df_train_data[idx2]')
            # display(df_train_data.iloc[idx2,-3:])
            last_filtered_value = filtered_value
            filtered_value = last_filtered_value + 0.004 * (
                df_train_data.iloc[idx2, -3:] - last_filtered_value
            )
            y = pd.DataFrame(columns=["Timestamp"])
            y = pd.concat(
                [y, pd.DataFrame([df_train_data.iloc[idx2, 0]], columns=["Timestamp"])],
                ignore_index=True,
            )
            #             print('display y :')
            #             display(y)
            #             print('end display y')

            #             print('display filtered value')
            #             display(pd.DataFrame(filtered_value).transpose())
            #             print('end display filtered value')
            df_allo.append(
                pd.concat([y, pd.DataFrame(filtered_value).transpose()], axis=1)
            )
        #             print('display df_allo')
        #             display(df_allo)

        # FIXME : change the name df_allo
        df_allo = pd.DataFrame(df_allo, columns=("Timestamp", "X", "Y", "Z"))

        df_allo.plot(x="Timestamp", legend=True, subplots=True, title="allo")
        stop()


#         v_candidate_x = pd.DataFrame({'Candidate':list(0)})
#         v_candidate_x = np.where(df_train_data.X.abs() <= df_treshold.X, 1, 0)